In [ ]:
!pip install nibabel
!pip install seaborn
!pip install openpyxl


In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution3D,Conv3D,Add,Concatenate, MaxPool3D,LeakyReLU,add, Convolution2D,BatchNormalization,AveragePooling3D, GlobalAveragePooling3D, ZeroPadding3D
from keras import backend as K
import cv2
import operator
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import glob
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.optimizers import SGD
import nibabel as nib
import pandas as pd
from scipy.ndimage import rotate
import random
import nibabel as nib
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.models import Model
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import numpy as np
import nibabel as nib
from matplotlib import animation, rc
from scipy.ndimage import rotate
import glob
import os
from sklearn.model_selection import KFold, StratifiedKFold,train_test_split
from scipy.ndimage import zoom
from matplotlib.patches import PathPatch, Rectangle
from IPython.display import HTML
from tensorflow.keras import layers
from sklearn.metrics import roc_curve, roc_auc_score
import matplotlib.pyplot as plt

In [ ]:
# Enable TPU
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print("Device:", tpu.master())
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
except:
    strategy = tf.distribute.get_strategy()
print("Number of accelerators: ", strategy.num_replicas_in_sync)

lidc load

In [ ]:
luna_df = pd.read_excel('/kaggle/input/spie-cropped-resampled-dataset/final_Luna16.xlsx')
column_luna = 'malignancy'
luna_series=luna_df['Series Uid'].tolist()

luna_folder = '/kaggle/input/spie-cropped-resampled-dataset/Luna_16_cropped_resampled/Luna_16_cropped_resampled'  
luna = []
for i in luna_series:
    nii_file_path = os.path.join(luna_folder, f'{i}.nii')  # Assuming NII files have the extension '.nii'
    
    if os.path.exists(nii_file_path):
        luna.append(i)
luna=np.array(luna)

print(luna.shape)





# Specify the path to the Excel file
file_path = '/kaggle/input/semi-testtrainval/test_initialtrain_vai_set.xlsx'
df = pd.read_excel(file_path)

z_train = df.iloc[:, 2]  

z_test_fixed = df.iloc[:, 0]  
nan_count = z_test_fixed.isna().sum()
z_test = z_test_fixed.dropna()

z_val_fixed = df.iloc[:, 1]  
nan_count_val = z_val_fixed.isna().sum()
z_val = z_val_fixed.dropna()


function for rotation

In [ ]:
BS= 68 # batch size
E = 200 #epoch

In [ ]:
def aug_rotate(x):

    rotated_slices = []
    for slice_idx in range(x.shape[2]):
        slice_data = x[:, :, slice_idx]  # Extract a single slice
        rotated_slice = rotate(slice_data, angle=angles[j], reshape=False, mode='nearest')
        rotated_slices.append(rotated_slice)

    # Create a new NIfTI image from the rotated slices
    rotated_img_data = np.stack(rotated_slices, axis=-1)
    # Create a new NIfTI image using the header information from the original image
    rotated_nifti = nib.Nifti1Image(rotated_img_data, img.affine)
    
    return rotated_nifti

split

In [ ]:
angles = [0, 45, 90, 135, 180, 225, 270, 315]



datas_test = []
labels_test = []
scans_test = []

datas_train = []
labels_train = []
scans_train = []

datas_val = []
labels_val = []
scans_val = []


for i in z_train:
    lab = luna_df.loc[luna_df['Series Uid'] == i, 'malignancy'].values[0]
    img_files = glob.glob(os.path.join(luna_folder, f"{i}.nii"))
    for img_file in img_files:
        img = nib.load(img_file)
        data = img.get_fdata()
        for j in range(len(angles)):
            rotated_nifti = aug_rotate(data)
            datas_train.append(rotated_nifti.get_fdata())
            labels_train.append(lab)
                    
for i in z_val:
    lab = luna_df.loc[luna_df['Series Uid'] == i, 'malignancy'].values[0]
    img_files = glob.glob(os.path.join(luna_folder, f"{i}.nii"))
    for img_file in img_files:
        img = nib.load(img_file)
        data = img.get_fdata()
        datas_val.append(data)
        labels_val.append(lab)

for i in z_test:
    lab = luna_df.loc[luna_df['Series Uid'] == i, 'malignancy'].values[0]
    img_files = glob.glob(os.path.join(luna_folder, f"{i}.nii"))
    for img_file in img_files:
        img = nib.load(img_file)
        data = img.get_fdata()
        datas_test.append(data)
        labels_test.append(lab)
            
X_train = np.array(datas_train)
y_train = np.array(labels_train)
X_test = np.array(datas_test)
y_test = np.array(labels_test)
X_val = np.array(datas_val)
y_val = np.array(labels_val)


X_train = np.array(datas_train)
y_train = np.array(labels_train)
X_test = np.array(datas_test)
y_test = np.array(labels_test)
X_val = np.array(datas_val)
y_val = np.array(labels_val)

print(z_train.shape, z_test.shape, z_val.shape)
print(X_train.shape, X_test.shape, X_val.shape)

model train

### train with 3D CNN

In [ ]:
def tdCNN(width=32, height=32, depth=32):
    inputs = tf.keras.Input((width, height, depth, 1))

    x = layers.Conv3D(filters=32, kernel_size=3, activation="relu", padding="same")(inputs)
    x = layers.MaxPool3D(pool_size=2)(x)
    x = layers.BatchNormalization()(x)
    
    x = layers.Conv3D(filters=64, kernel_size=3, activation="relu", padding="same")(inputs)
    x = layers.MaxPool3D(pool_size=2)(x)
    x = layers.BatchNormalization()(x)

    x = layers.Conv3D(filters=64, kernel_size=3, activation="relu", padding="same")(x)
    x = layers.MaxPool3D(pool_size=2)(x)
    x = layers.BatchNormalization()(x)

    x = layers.Conv3D(filters=128, kernel_size=3, activation="relu", padding="same")(x)
    x = layers.MaxPool3D(pool_size=2)(x)
    x = layers.BatchNormalization()(x)

    x = layers.Conv3D(filters=256, kernel_size=3, activation="relu", padding="same")(x)
    x = layers.MaxPool3D(pool_size=2)(x)
    x = layers.BatchNormalization()(x)

    x = layers.GlobalAveragePooling3D()(x)
    x = layers.Dense(units=512, activation="relu")(x)
    x = layers.Dropout(0.4)(x)

    outputs = layers.Dense(units=1, activation="sigmoid")(x)

    model = tf.keras.Model(inputs, outputs, name="3dcnn")
    return model


In [ ]:
with strategy.scope():
    model=tdCNN()
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

lr_callback = tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.001,
        patience=5,
        min_lr=1e-6)

# Define the model checkpoint callback to save the best model
checkpoint_callback = ModelCheckpoint('3DCNN.h5', monitor='val_accuracy',save_best_only=True)

BATCH_SIZE_TPU = BS * strategy.num_replicas_in_sync
# Fit data to model
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=E, batch_size=BATCH_SIZE_TPU,callbacks=[lr_callback,checkpoint_callback],verbose=0)


# Evaluate the model - report accuracy and capture it into a list for future reporting
best_model1 = tf.keras.models.load_model('3DCNN.h5')

loss1, accuracy1 = best_model1.evaluate(X_test, y_test)   


#----------------------------------------------------------------
predictions1 = best_model1.predict(X_test)
predicted_classes = (predictions1 > 0.5).astype(int)

# Generate the classification report
class_names = list(['benign','malignant'])
report = classification_report(y_test, predicted_classes , target_names=class_names,digits=5)
print("Classification Report:\n", report)
auc1 = roc_auc_score(y_test, predictions1)
print(f'AUC: {auc1}')

# Generate the confusion matrix
cm = confusion_matrix(y_test, predicted_classes )
plt.figure(figsize=(5,4))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()
#----------------------------------------------------------------

print(f'Accuracy: {accuracy1*100}')
print(f'loss: {loss1*100}')




### train with resnet

In [ ]:
from tensorflow.keras.layers import Input, MaxPooling3D, Add, AveragePooling3D
def resnet_block(x, filters, num_blocks):
    
    if filters!=64:
        shortcut = Conv3D(filters, (1, 1,1), strides=(1, 1,1), padding='same',activation='relu')(x)
        x = Conv3D(filters,(3,3,3), padding='same',activation='relu')(x)
        x = Conv3D(filters,(3,3,3), padding='same',activation='relu')(x)
        x = BatchNormalization()(x)
        x=Add()([x, shortcut])
        num_blocks=num_blocks-1
    
    for i in range(num_blocks):
        input=x
        x = Conv3D(filters,(3,3,3), padding='same',activation='relu')(x)
        x = Conv3D(filters,(3,3,3), padding='same',activation='relu')(x)
        x = BatchNormalization()(x)
        x=Add()([x, input])
    
    return x


def resnet_34(input_shape=(32,32,32,1)):
    
    inputs = Input(shape=input_shape)
    filters = [64, 128, 256, 512]
    block_sizes = [3, 4, 6, 5]

    
    # Initial convolutional layer
    
    x = Conv3D(64, (7, 7,7), strides=(2, 2,2),padding='same',activation='relu')(inputs)
    x = MaxPooling3D(pool_size=(3, 3,3), strides=(2, 2,2), padding='same')(x)

    for i, num_blocks in enumerate(block_sizes):
        x = resnet_block(x, filters[i], num_blocks)
    

    x = AveragePooling3D(pool_size=(2, 2, 2))(x)
    x = Flatten()(x)
    outputs = Dense(1, activation='sigmoid')(x)

    model = Model(inputs=inputs, outputs=outputs)
    return model


model = resnet_34()

In [ ]:
with strategy.scope():

    model=resnet_34(input_shape=(32,32,32,1))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


lr_callback = tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.001,
        patience=5,
        min_lr=1e-6)

# Define the early stopping callback to stop training if validation loss does not improve

checkpoint_callback = ModelCheckpoint('resnet.h5', monitor='val_accuracy',save_best_only=True)
BATCH_SIZE_TPU = BS * strategy.num_replicas_in_sync

# Fit data to model
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=E, batch_size=BATCH_SIZE_TPU,
                    callbacks=[checkpoint_callback,lr_callback],verbose=0)
    

# Evaluate the model - report accuracy and capture it into a list for future reporting
best_model = tf.keras.models.load_model('resnet.h5')
loss2, accuracy2 = best_model.evaluate(X_test, y_test)


#----------------------------------------------------------------
predictions2 = best_model.predict(X_test)
predicted_classes = (predictions2 > 0.5).astype(int)

# Generate the classification report
class_names = list(['benign','malignant'])
report = classification_report(y_test, predicted_classes , target_names=class_names,digits=5)
print("Classification Report:\n", report)
auc2 = roc_auc_score(y_test, predictions2)
print(f'AUC: {auc2}')


# Generate the confusion matrix
cm = confusion_matrix(y_test, predicted_classes )
plt.figure(figsize=(5,4))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()
#----------------------------------------------------------------



print(f'Accuracy: {accuracy2*100}')
print(f'loss: {loss2*100}')

### train with LCCNet

In [ ]:
def lcc():
    model = Sequential()
    model.add(Conv3D(32,(3,3,3), activation='relu', kernel_initializer='he_uniform', padding='same',input_shape=(32,32,32,1)))
    model.add(BatchNormalization())
    model.add(Conv3D(32, (3, 3,3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPool3D((2, 2,2)))
    #model.add(Dropout(0.1))
    
    model.add(Conv3D(64, (3, 3,3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(BatchNormalization())
    model.add(Conv3D(64, (3, 3,3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPool3D((2, 2,2)))
    #model.add(Dropout(0.2))
    
    model.add(Conv3D(128, (3, 3,3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(BatchNormalization())
    model.add(Conv3D(128, (3, 3,3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPool3D((2, 2,2)))
    model.add(Dropout(0.3))
    
    model.add(Conv3D(128, (3, 3,3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(BatchNormalization())
    model.add(Conv3D(128, (3, 3,3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(BatchNormalization())
    model.add(Conv3D(128, (3, 3,3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(BatchNormalization())
    model.add(Conv3D(128, (3, 3,3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPool3D((2, 2,2)))

    model.add(Conv3D(256, (3, 3,3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(BatchNormalization())
    model.add(Conv3D(256, (3, 3,3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(BatchNormalization())
    #model.add(MaxPool3D((2, 2,2))
    model.add(Dropout(0.5))
    
    model.add(Flatten())
    model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
    model.add(BatchNormalization())
    model.add(Dense(64, activation='relu', kernel_initializer='he_uniform'))
    model.add(BatchNormalization())
    #model.add(Dropout(0.5))
    model.add(Dense(32, activation='relu', kernel_initializer='he_uniform'))
    model.add(BatchNormalization())
    model.add(Dense(8, activation='relu', kernel_initializer='he_uniform'))


    model.add(Dense(1, activation='sigmoid', kernel_initializer='he_uniform'));
    return model

In [ ]:
best_model = tf.keras.models.load_model('/kaggle/input/semi-testtrainval/semiSupervised_HU1.h5')
best_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
loss3, accuracy3 = best_model.evaluate(X_test, y_test)


#----------------------------------------------------------------
predictions3 = best_model.predict(X_test)
predicted_classes = (predictions3 > 0.5).astype(int)

# Generate the classification report
class_names = list(['benign','malignant'])
report = classification_report(y_test, predicted_classes , target_names=class_names)
print("Classification Report:\n", report)
auc3 = roc_auc_score(y_test, predictions3)
print(f'AUC: {auc3}')


# Generate the confusion matrix
cm = confusion_matrix(y_test, predicted_classes )
plt.figure(figsize=(5,4))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()
#----------------------------------------------------------------

print(f'Accuracy: {accuracy3*100}')
print(f'loss: {loss3*100}')

In [ ]:
print("Accuracy")
print(f'model1(3D CNN)  : {accuracy1*100}')
print(f'model2(RESNET34): {accuracy2*100}')
print(f'model3(LCC-Net) : {accuracy3*100}')

print("------------------------------")
print("Loss")
print(f'model1(3D CNN)  : {loss1*100}')
print(f'model2(RESNET34): {loss2*100}')
print(f'model3(LCC-Net) : {loss3*100}')

print("------------------------------")
print("AUC")
print(f'model1(3D CNN)  : {auc1*100}')
print(f'model2(RESNET34): {auc2*100}')
print(f'model3(LCC-Net) : {auc3*100}')

In [ ]:
fpr1, tpr1, _ = roc_curve(y_test, predictions1)
fpr2, tpr2, _ = roc_curve(y_test, predictions2)
fpr3, tpr3, _ = roc_curve(y_test, predictions3)


# Plot the ROC curves
plt.figure(figsize=(8, 6))
plt.plot(fpr1, tpr1, color='blue', label=f'Model 1 ROC curve (AUC = {auc1:.5f})')
plt.plot(fpr2, tpr2, color='green', label=f'Model 2 ROC curve (AUC = {auc2:.5f})')
plt.plot(fpr3, tpr3, color='orange', label=f'Model 3 ROC curve (AUC = {auc3:.5f})')
plt.plot([0, 1], [0, 1], color='red', linestyle='--')  # Diagonal line representing random chance
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curves')
plt.legend(loc='lower right')
plt.show()

In [ ]:
# Assuming correct_benign_indices contains all the correct benign predictions
correct_indices = np.where(predicted_classes == y_test)[0]
correct_benign_indices = [i for i in correct_indices if y_test[i] == 0]

selected_indices = []
unique_z_test_values = set()
for i in range(40):
    index = 15 + i * 32
    if index < len(correct_benign_indices):  # Ensure we don't go out of bounds
        z_value = z_test[correct_benign_indices[index]]
        if z_value not in unique_z_test_values:  # Check if the z_test value is unique
            selected_indices.append(correct_benign_indices[index])
            unique_z_test_values.add(z_value)

# Print the corresponding z_test values
print("Plotted unique z_test values:")
for idx in selected_indices:
    print(z_test[idx])

# Plotting the selected benign CT images
plt.figure(figsize=(60, 100))
for i, idx in enumerate(selected_indices):
    plt.subplot(10, 4, i + 1)  
    plt.imshow(X_test[idx].reshape(32, 32, 32)[16], cmap='gray')  # Adjusted to slice 16
    plt.title(f'Benign (Index: {idx})')
    plt.axis('off')

plt.suptitle('Selected Correctly Predicted Benign CT Images with Unique z_test Values')
plt.show()


In [ ]:
# Assuming correct_malignant_indices contains all the correct malignant predictions
correct_indices = np.where(predicted_classes == y_test)[0]
correct_malignant_indices = [i for i in correct_indices if y_test[i] == 1]

# Select indices based on the pattern 15, 15+32, 15+32+32, ...
selected_indices = []
unique_z_test_values = set()
for i in range(40):  # To get up to 40 plots
    index = 15 + i * 32
    if index < len(correct_malignant_indices):  # Ensure we don't go out of bounds
        z_value = z_test[correct_malignant_indices[index]]
        if z_value not in unique_z_test_values:  # Check if the z_test value is unique
            selected_indices.append(correct_malignant_indices[index])
            unique_z_test_values.add(z_value)

# Print the corresponding z_test values
print("Plotted unique z_test values:")
for idx in selected_indices:
    print(z_test[idx])

# Plotting the selected malignant CT images
plt.figure(figsize=(60, 100))
for i, idx in enumerate(selected_indices):
    plt.subplot(10, 4, i + 1)  # 10 rows and 5 columns for 50 plots
    plt.imshow(X_test[idx].reshape(32, 32, 32)[16], cmap='gray')  # Adjusted to slice 16
    plt.axis('off')

plt.suptitle('Selected Correctly Predicted Malignant CT Images with Unique z_test Values')
plt.show()
